### Comparaison sémantique

(Basé sur la technique du paper Sentence Similarity Techniques for Short vs Variable Length Text using Word Embeddings)

### Test avec tout 2020 (tjrs avec juste quelques expressions test, vu qu'on a pas choisi lesquelles on garde)

#### Install and import every packages that will be necessary for the analysis

In [16]:
!pip install pickle-mixin
!pip install pyarrow
!pip install pathlib


import pyarrow
import pandas as pd
import numpy as np
import pathlib
from datetime import datetime
import sys
import bz2
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

### Imports, et import du pretrained model qu'on utilise ( pour l'instant all-MiniLM-L6-v2 me semble le mieux)

In [1]:
#si besoin
#pip install -U sentence-transformers
#pip install gensim

In [17]:
#NLP libraries
import nltk
from nltk import word_tokenize
#from gensim.models import FastText #pas utilisé finalement

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer(r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\pretrained_model') #pour si la ligne d'en dessous ne marche pas, telecharger le model en local et mettre le path
model = SentenceTransformer('all-MiniLM-L6-v2') #import of the pretrained model we will use


# j'ai testé plysieurs models:

# all-MiniLM-L6-v2: ça passe, treshold à 0.6 ?
#'bert-base-nli-mean-tokens' : deprecated
#all-mpnet-base-v2: ne se télécharge pas
#multi-qa-mpnet-base-dot-v1: ne va pas, tous les scores sont trés haut meme pour 'truc contraires
#multi-qa-distilbert-cos-v1: mouai


#### create a dataframe de test with the expression we want to compare, and their score (remplacer aprés avec le vrai)

In [19]:
#create a dataframe with the ewpression we want to compare, and their score
expressions = pd.DataFrame({'expression': ["I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"], 'expression_score': [2.91, 3.02, 3.34, 3.48, 6.02, 6.45]})
expressions.head(6)

,expression,expression_score
0,I'm not sure,2.91
1,I dont know,3.02
2,I suppose,3.34
3,I think,3.48
4,I'm sure,6.02
5,I know,6.45


### Comparaison des citations (morceau par morceau) aux expressions du paper, et création d'un dataframe avec qids, quote, cosine_similarities.

Je decoupe les citations. Pour chaque expression: je compare chaque morceau de la citation avec l'expression, ça donne un score de similarité pour chaque morceau: on prend le max (ou bien faire un teshold ? à voir).  







In [22]:
path_to_file = r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\quotes-processed.json.bz2' #path florette

i = 0

with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=10, orient = 'records') as df_reader:
    for quotes in df_reader:
        #First, we separate the word and symbol using tokenisation. This will allow us to then reform n_grams with the desired number of words/symbol.
        quotes['n_grams'] = quotes['quotation'].apply(lambda x: nltk.word_tokenize(x))

        #Then reform chunk of n words
        n = 4 #number in n_gram
        quotes['n_grams'] = quotes['n_grams'].apply(lambda x: (list(" ".join(x[i : i+n-1]) for i in range(len(x) - n + 1))))
        
        #coosine similarity
        quotes['cosine_similarities'] = quotes['n_grams'].apply(lambda x: list(max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x)).flatten()[0] for expression_embedded in model.encode(expressions.expression)))
        
        i = i+1
        
        if i >= 1 : break
            
            
#df_reader = pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=100000, orient = 'records')

In [23]:
quotes.head()

,speaker,qids,quotation,size,n_grams,cosine_similarities
0,Sue Myrick,[Q367796],Department of Homeland Security was livid and ...,14,"[Department of Homeland, of Homeland Security,...","[0.3032319, 0.3031956, 0.23602816, 0.22850059,..."
1,Meghan King Edmonds,[Q20684375],I met them when they just turned 4 and 7. They...,19,"[I met them, met them when, them when they, wh...","[0.2303027, 0.2367206, 0.27510583, 0.29699588,..."
2,Dexter Smith,[Q5268447],The delay will have an impact on Slough but th...,37,"[The delay will, delay will have, will have an...","[0.41248465, 0.34659547, 0.50099874, 0.6188506..."
3,Barry Coppinger,[Q4864119],The scheme treats addiction as an illness and ...,16,"[The scheme treats, scheme treats addiction, t...","[0.27248415, 0.21348144, 0.23313306, 0.2357903..."
4,Ben Carson,[Q816459],These actions will allow households who have a...,27,"[These actions will, actions will allow, will ...","[0.2834597, 0.28215024, 0.22757119, 0.18103966..."


je voulais mettre les scores des differentes expressions dans differentes colonnes, mais j'y arrive pas. Comme je sais pas si ça ser plus pratique pour la suite tout dans une meme colonne ou dans differentes colonnes je laisse comma ça en attendant.

## Probleme à résoudre:
on va choisir un treshold pour l'aggregation des scores. Le probleme, c'est que là dans mon code on choisit le nombre de mots dans les n grams (n), on fait les n grams, puis ont fait la comparaison avec les expressions. Le truc c'est que on a des expressions de 2 à 5 mots qu'on compare avec des n grams tous de n mots, et ça pourrait se voir sur les score et la precision du trashold (genre idealement ça aurait peut etre été mieux d'adapter n à chaque expression, mais ça veut dire recalculer les ngrams pour chaque citations, j'ai peur que ça soit monstrueux et que ça rallong enormement...). Je ferais quelques test pour voir si ça gene pour le treshold.

### Pour tester direct sur un petit dataframe

#### Extract a small part of the whole quotes file to test

In [10]:
#Extract a small part of the whole file to test 
path = r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\quotes-processed.json.bz2' #path florette
quotes = pd.DataFrame(columns=('qids','quotation'))
i = 0
with bz2.open(path, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) 
    #print(instance)
    #print(instance['quotation'])
    i = i + 1
    quotes = quotes.append({'qids': instance['qids'], 'quotation': instance['quotation']}, ignore_index=True)
    #print(i)
    if i == 10:
      break

#### append some quotation for testing the efficacy

In [6]:
quotes = quotes.append({'qids': 'Qtest1', 'quotation': 'I know it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest2', 'quotation': 'I dont know if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest3', 'quotation': 'I am sure it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest4', 'quotation': 'I am not sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest5', 'quotation': 'I dont eat an apple'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest6', 'quotation': 'I pretty sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest7', 'quotation': 'I was sure if it is a fool'}, ignore_index=True)


#### Create the n grams

In [ ]:
#First, we separate the word and symbol using tokenisation. This will allow us to then reform n_grams with the desired number of words/symbol.
        quotes['n_grams'] = quotes['quotation'].apply(lambda x: nltk.word_tokenize(x))

        #Then reform chunk of n words
        n = 4 #number in n_gram
        quotes['n_grams'] = quotes['n_grams'].apply(lambda x: (list(" ".join(x[i : i+n-1]) for i in range(len(x) - n + 1))))

#### Comparison

In [ ]:
#coosine similarity
        quotes['cosine_similarities'] = quotes['n_grams'].apply(lambda x: list(max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x)).flatten()[0] for expression_embedded in model.encode(expressions.expression)))